Run `docker-compose up -d` to start the services.

In [1]:
import json
import random


def generate_data(num_users=100):
    users = []
    for i in range(1, num_users + 1):
        user = {
            "id": i,
            "name": f"User{i}",
            "friends": []
        }
        users.append(user)

    for user in users:
        num_friends = random.randint(1, num_users // 2)
        friends = random.sample([u["id"] for u in users if u["id"] != user["id"]], num_friends)
        user["friends"] = friends

    data = {
        "users": users
    }

    return data


data = generate_data(100)

with open("data.json", "w") as f:
    json.dump(data, f, indent=2)

print("data.json file has been generated.")

data.json file has been generated.


In [2]:
import pymongo
import redis
from neo4j import GraphDatabase

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]
collection = db["users"]

r = redis.Redis(host='localhost', port=6379, db=0)

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [3]:
with open("data.json") as file:
    data = json.load(file)

collection.insert_many(data["users"])

InsertManyResult([ObjectId('6659982fe37981682a4d2d78'), ObjectId('6659982fe37981682a4d2d79'), ObjectId('6659982fe37981682a4d2d7a'), ObjectId('6659982fe37981682a4d2d7b'), ObjectId('6659982fe37981682a4d2d7c'), ObjectId('6659982fe37981682a4d2d7d'), ObjectId('6659982fe37981682a4d2d7e'), ObjectId('6659982fe37981682a4d2d7f'), ObjectId('6659982fe37981682a4d2d80'), ObjectId('6659982fe37981682a4d2d81'), ObjectId('6659982fe37981682a4d2d82'), ObjectId('6659982fe37981682a4d2d83'), ObjectId('6659982fe37981682a4d2d84'), ObjectId('6659982fe37981682a4d2d85'), ObjectId('6659982fe37981682a4d2d86'), ObjectId('6659982fe37981682a4d2d87'), ObjectId('6659982fe37981682a4d2d88'), ObjectId('6659982fe37981682a4d2d89'), ObjectId('6659982fe37981682a4d2d8a'), ObjectId('6659982fe37981682a4d2d8b'), ObjectId('6659982fe37981682a4d2d8c'), ObjectId('6659982fe37981682a4d2d8d'), ObjectId('6659982fe37981682a4d2d8e'), ObjectId('6659982fe37981682a4d2d8f'), ObjectId('6659982fe37981682a4d2d90'), ObjectId('6659982fe37981682a4d2d

In [4]:
def create_user(tx, user_id, name):
    tx.run("CREATE (u:User {id: $user_id, name: $name})", user_id=user_id, name=name)


def create_friendship(tx, user_id, friend_id):
    tx.run("""
        MATCH (u1:User {id: $user_id}), (u2:User {id: $friend_id})
        MERGE (u1)-[:FRIEND]->(u2)
        """, user_id=user_id, friend_id=friend_id)


with driver.session() as session:
    for user in data["users"]:
        session.execute_write(create_user, user["id"], user["name"])
        for friend_id in user["friends"]:
            session.execute_write(create_friendship, user["id"], friend_id)

In [5]:
def get_graph(tx):
    result = tx.run("MATCH (u:User)-[:FRIEND]-(f:User) RETURN u.name, f.name")
    for record in result:
        print(f"{record['u.name']} is friends with {record['f.name']}")


with driver.session() as session:
    session.execute_read(get_graph)

User95 is friends with User1
User94 is friends with User1
User92 is friends with User1
User85 is friends with User1
User82 is friends with User1
User78 is friends with User1
User75 is friends with User1
User73 is friends with User1
User71 is friends with User1
User70 is friends with User1
User63 is friends with User1
User55 is friends with User1
User52 is friends with User1
User49 is friends with User1
User44 is friends with User1
User41 is friends with User1
User38 is friends with User1
User37 is friends with User1
User31 is friends with User1
User30 is friends with User1
User23 is friends with User1
User13 is friends with User1
User12 is friends with User1
User6 is friends with User1
User2 is friends with User1
User100 is friends with User2
User95 is friends with User2
User90 is friends with User2
User89 is friends with User2
User86 is friends with User2
User85 is friends with User2
User84 is friends with User2
User72 is friends with User2
User64 is friends with User2
User63 is frien

In [6]:
from bson import ObjectId


class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, ObjectId):
            return str(obj)
        return super(JSONEncoder, self).default(obj)


def get_user_from_mongo(user_id):
    cache_key = f"user:{user_id}"
    cached_user = r.get(cache_key)

    if cached_user:
        return json.loads(cached_user)

    user = collection.find_one({"id": user_id})

    if user:
        r.set(cache_key, json.dumps(user, cls=JSONEncoder), ex=3600)

    return user


for x in range(1, 101):
    user_data = get_user_from_mongo(x)
    print(user_data)

{'_id': ObjectId('6659982fe37981682a4d2d78'), 'id': 1, 'name': 'User1', 'friends': [40, 100, 73, 32, 58]}
{'_id': ObjectId('6659982fe37981682a4d2d79'), 'id': 2, 'name': 'User2', 'friends': [70, 91, 26, 45, 41, 69, 1, 79, 6, 73, 72, 37, 74, 16, 42, 71, 8, 35, 81, 13, 56, 48, 17, 3, 78, 98, 88, 23, 75, 12, 76, 54]}
{'_id': ObjectId('6659982fe37981682a4d2d7a'), 'id': 3, 'name': 'User3', 'friends': [6, 77, 35, 5, 87, 42, 94]}
{'_id': ObjectId('6659982fe37981682a4d2d7b'), 'id': 4, 'name': 'User4', 'friends': [7, 51, 76, 21, 88, 86, 26, 38, 62, 45, 29, 11, 34, 58, 23, 50, 46, 55, 27, 65, 53, 30, 41, 69, 22]}
{'_id': ObjectId('6659982fe37981682a4d2d7c'), 'id': 5, 'name': 'User5', 'friends': [89, 26, 17, 84, 29, 27, 98, 90, 18, 21, 8, 58, 3, 37]}
{'_id': ObjectId('6659982fe37981682a4d2d7d'), 'id': 6, 'name': 'User6', 'friends': [82, 96, 1, 93, 48, 59, 88, 86, 69, 23, 55, 5, 50, 40, 66, 9, 61, 87, 98, 62, 2, 68, 81, 38, 52, 97, 77, 32, 73, 75, 31, 17, 30, 20, 14, 65, 42, 85, 63, 92, 13, 21, 72,

In [7]:
def get_graph_with_cache():
    cache_key = "graph:friendships"
    cached_graph = r.get(cache_key)

    if cached_graph:
        return json.loads(cached_graph)

    graph_data = []

    with driver.session() as session:
        def fetch_graph(tx):
            result = tx.run("MATCH (u:User)-[:FRIEND]->(f:User) RETURN u.name, f.name")
            for record in result:
                graph_data.append({"user": record['u.name'], "friend": record['f.name']})

        session.execute_read(fetch_graph)

    r.set(cache_key, json.dumps(graph_data), ex=3600)

    return graph_data


graph_data = get_graph_with_cache()
print(graph_data)

[{'user': 'User95', 'friend': 'User1'}, {'user': 'User94', 'friend': 'User1'}, {'user': 'User92', 'friend': 'User1'}, {'user': 'User85', 'friend': 'User1'}, {'user': 'User82', 'friend': 'User1'}, {'user': 'User78', 'friend': 'User1'}, {'user': 'User75', 'friend': 'User1'}, {'user': 'User73', 'friend': 'User1'}, {'user': 'User71', 'friend': 'User1'}, {'user': 'User70', 'friend': 'User1'}, {'user': 'User63', 'friend': 'User1'}, {'user': 'User55', 'friend': 'User1'}, {'user': 'User52', 'friend': 'User1'}, {'user': 'User49', 'friend': 'User1'}, {'user': 'User44', 'friend': 'User1'}, {'user': 'User41', 'friend': 'User1'}, {'user': 'User38', 'friend': 'User1'}, {'user': 'User37', 'friend': 'User1'}, {'user': 'User31', 'friend': 'User1'}, {'user': 'User30', 'friend': 'User1'}, {'user': 'User23', 'friend': 'User1'}, {'user': 'User13', 'friend': 'User1'}, {'user': 'User12', 'friend': 'User1'}, {'user': 'User6', 'friend': 'User1'}, {'user': 'User2', 'friend': 'User1'}, {'user': 'User100', 'frien

In [8]:
def clear_redis():
    r = redis.Redis(host='localhost', port=6379, db=0)
    r.flushdb()
    print("Redis database has been flushed")


def clear_mongodb():
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["mydatabase"]
    db.drop_collection("users")
    print("MongoDB 'users' collection has been dropped")


def clear_neo4j():
    uri = "bolt://localhost:7687"
    username = "neo4j"
    password = "password"
    driver = GraphDatabase.driver(uri, auth=(username, password))

    def clear_database(tx):
        tx.run("MATCH (n) DETACH DELETE n")

    with driver.session() as session:
        session.execute_write(clear_database)
    print("Neo4J database has been cleared")


clear_redis()
clear_mongodb()
clear_neo4j()

Redis database has been flushed
MongoDB 'users' collection has been dropped
Neo4J database has been cleared
